# 聚合数据

分组操作首先拆分数据，然后对分组数据应用函数，最后联合结果。在 Pandas 中，使用`groupby()`函数进行分组，返回`GroupBy`对象。`GroupBy`对象已经定义了一些聚合方法，例如常见的描述性统计函数（求和、求均值）等，用来聚合数据。不过，很多场景需要使用自定义的聚合操作。

使用`GroupBy`对象的`agg`或`apply`等方法，传递自定义函数，就可以实现自定义的聚合操作。本节介绍`agg()`或`apply()`等的使用说明。

In [1]:
import numpy as np
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

## 使用`agg()`聚合数据

`agg`与`aggregate`功能一样，其使用语法为：
```python
g1.agg(func_or_funcs, *args, **kwargs)```
主要参数
- `func_or_funcs`，函数，字符串，字典或列表。
- `*args`, 传入函数的位置参数。
- `**kwargs`, 传入函数的关键字位置参数。

下面创建一个`DataFrame`对象：

In [2]:
dict_obj = {'key1' : ['a', 'b', 'a', 'b', 
                      'a', 'b', 'a', 'a'],
            'key2' : ['one', 'one', 'two', 'three',
                      np.nan, 'two', 'one', 'three'],
            'data1': np.random.randint(1, 10, 8),
            'data2': np.random.randint(1, 10, 8),
            'data3': ['1', '2', '3', '4', '5', '6', '7', '8']}
df = pd.DataFrame(dict_obj)
df

,key1,key2,data1,data2,data3
0,a,one,8,9,1
1,b,one,3,5,2
2,a,two,8,7,3
3,b,three,4,6,4
4,a,NaN,6,7,5
5,b,two,6,2,6
6,a,one,2,7,7
7,a,three,8,6,8


使用`agg`方法来实现自定义数据聚合，下面传入一个一个匿名函数：

In [3]:
# 应用自定义聚合函数
df.groupby('key1').agg(lambda x: x.max() - x.min())

,data1,data2
key1,,
a,6,3
b,3,4


注意，匿名函数无法应用与`data3`列。下面通过元组提供多个聚合函数：

In [4]:
# 应用自定义聚合函数
df.groupby('key1').agg(['mean', 'std', 'count', ('range', lambda df: df.max() - df.min())])

data1                           data2                      
          mean       std count range      mean       std count range
key1                                                                
a     6.400000  2.607681     5     6  7.200000  1.095445     5     3
b     4.333333  1.527525     3     3  4.333333  2.081666     3     4

在聚合数据时，使用字典来指定不同列应用不用聚合函数：

In [5]:
# 不同列应用不同聚合函数
col2funcs = {
    'data1': ['count', 'mean'], 
    'data2': ['count', 'sum'], 
    'data3': [('str_join', lambda s: '_'.join(s))]}
df.groupby('key1').agg(col2funcs)

data1           data2          data3
     count      mean count sum   str_join
key1                                     
a        5  6.400000     5  36  1_3_5_7_8
b        3  4.333333     3  13      2_4_6

In [6]:
ser = pd.Series(range(6))
ser
keys = ['A', 'B', 'C', 'A', 'B', 'C']
# 使用keys进行分组
g1 = ser.groupby(keys)


0    0
1    1
2    2
3    3
4    4
5    5
dtype: int64

## 使用`apply()`聚合数据

使用`agg()`把一个函数应用到一个数据列上。Pandas 分组运算还提供`apply()`方法，用于泛化的聚合操作。`apply()`应用到一个数据块上，可以返回标量值，以及数组等。

下面定义一个要传入`apply()`的函数。函数第一个参数必须是`DataFrame`对象，可以返回标量、序列对象或`DataFrame`数组：

In [7]:
def f1(df):
    xmax = max(df.data1.max(), df.data2.max())
    xmin = min(df.data1.min(), df.data2.min())
    xs = '_'.join(df.data3)
    return pd.Series([xmin, xmax, xmax-xmin, xs], index=['xmin', 'xmax', 'maxmin', 'xs'])

In [8]:
df.groupby('key1').apply(f1)

,xmin,xmax,maxmin,xs
key1,,,,
a,2,9,7,1_3_5_7_8
b,2,6,4,2_4_6


## 使用`transform()`聚合数据

Pandas 还可以使用`transform()`方法实现分组转换，与`apply()`方法类似，但限制更多：
- 可以产生一个标量值
- 可以产生一个与输入分组数据维度相同的对象
- 不能改变输入数组

例如下面定义一个归一化函数，并进行分组转换：

In [9]:
def normalize(x):
    return (x - x.mean()) / x.std()

# 分组转换
df.groupby('key1').transform(normalize)

,data1,data2
0,0.613572,1.643168
1,-0.872872,0.320256
2,0.613572,-0.182574
3,-0.218218,0.800641
4,-0.153393,-0.182574
5,1.091089,-1.120897
6,-1.687323,-0.182574
7,0.613572,-1.095445


## 使用`filter()`过滤数据

Pandas 还提供了`filter()`来聚合数据，其主要用于过滤数据：

In [10]:
# 过滤
df.groupby(['key1', 'key2']).filter(lambda x: len(x) > 1)

,key1,key2,data1,data2,data3
0,a,one,8,9,1
6,a,one,2,7,7
